# Feedback Prize - Evaluating Student Writing
#### Analyze argumentative writing elements from students grade 6-12
![](https://storage.googleapis.com/kaggle-competitions/kaggle/31779/logos/header.png?t=2021-11-12-22-52-17&quot)

https://drive.google.com/file/d/1r9vxKn5Az3ZBoZ7PgkmIh2ov87axyN8B/view

# Import packages

In [ ]:
# Install packages

In [ ]:
# Import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
from tqdm import tqdm
tqdm.pandas()
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import torch
import torch.nn as nn
from joblib import Parallel, delayed
from transformers import AutoModelForTokenClassification
from transformers import AutoModel, AutoTokenizer, AutoConfig, TrainingArguments, Trainer, EvalPrediction
import sys
import time
import gc
import pickle
import re
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import sklearn
SEED = 42

# Modelling - NER

In [ ]:
tag2idx = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 2,
    "I-Position": 3,
    "B-Evidence": 4,
    "I-Evidence": 5,
    "B-Claim": 6,
    "I-Claim": 7,
    "B-Concluding Statement": 8,
    "I-Concluding Statement": 9,
    "B-Counterclaim": 10,
    "I-Counterclaim": 11,
    "B-Rebuttal": 12,
    "I-Rebuttal": 13,
    "O": 14
}


idx2tag = {v: k for k, v in tag2idx.items()}

In [ ]:
def tokenize_sentence(x):
    "takes in a string and returns tokenized list"

    return [x for x in x.strip().lower().split(" ") if len(x) > 0]

In [ ]:
def get_tags(sent, rec_id):
    '''
    Input: sent as a sentence tokenized as list of tokens
    output: tags 
    '''
    sent = [x for x in sent]
    data = ' '.join(sent)
    
    train_data = pd.DataFrame(data = None, columns = ['id'])
    temp = train_data[train_data['id']==rec_id].reset_index(drop = True)
    all_tags = []
    correct_tags = []
    
    if temp.shape[0] > 0:
        for j in range(0, len(temp)):     
            pred_string = temp['discourse_text'].iloc[j]
            pred_string = re.sub('\n+', ' ', pred_string).strip().replace("\xa0"," ").lower()
            discourse_type = temp['discourse_type'].iloc[j]
            ep = tokenize_sentence(pred_string)

            start_index = data.find(pred_string)
            stop_index = data.find(pred_string) + len(pred_string)
            tokens_for_tagging = data[start_index:stop_index].split(' ')

            i = 0
            tag = []
            tag_flag = False
            while i < len(sent):
                if (start_index == 0) & (i == 0) :
                    tag.append('B-' + discourse_type)
                    i+=1

                    tag = tag + ['I-' + discourse_type] * (len(ep)-1)
                    i+=(len(ep)-1)
                    tag_flag = True
                elif (data.split(' ')[i] == tokens_for_tagging[0]) & (len(' '.join(data.split(' ')[0:i]))+1 == start_index) & (not tag_flag):
                    tag.append('B-' + discourse_type)
                    i+=1

                    tag = tag + ['I-' + discourse_type] * (len(ep)-1)
                    i+=(len(ep)-1)
                else:
                    tag.append('O')                
                    i+=1
            all_tags.append(tag)
            # Update text to mask already tagged tokens
            data = ' '.join(['masked_token' if (('B' in tag[x]) or ('I' in tag[x])) else data.split(' ')[x] for x in range(0, len(data.split(' ')))])

        for i in range(0, len(all_tags[0])):
            sum_O_tags = 0
            tags = []
            for j in range(0, len(all_tags)):
                if all_tags[j][i] == 'O':
                    sum_O_tags += 1
                else:
                    tags.append(all_tags[j][i])
            if sum_O_tags == len(all_tags):
                correct_tags.append('O')
            else:
                if len(tags)>1:
                    print('More than 1 tag for a token in record_id', rec_id)
                    print(tags)
                correct_tags.append(tags[0])

    else:
        correct_tags = ['O'] * len(sent)
    return correct_tags

In [ ]:
def vocab_sent_tokenize_label(sent_tokenized, token_tag):
    try:
        vocab_sent_token = []
        sent_input_ids = []
        vocab_token_tag = []
        token_tag_ids = []
        for sent_token_, tag_ in zip(sent_tokenized, token_tag):
            _vocab_sent_token = tokenizer.tokenize(sent_token_)
            _sent_input_ids = [
                tokenizer.convert_tokens_to_ids(x) for x in _vocab_sent_token
            ]
            _vocab_token_tag = [tag_] * len(_vocab_sent_token)
            _token_tag_ids = [tag2idx[x] for x in _vocab_token_tag]

            vocab_sent_token.extend(_vocab_sent_token)
            sent_input_ids.extend(_sent_input_ids)
            vocab_token_tag.extend(_vocab_token_tag)
            token_tag_ids.extend(_token_tag_ids)
        return vocab_sent_token, sent_input_ids, vocab_token_tag, token_tag_ids
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
def sent_tag_tokenization(data):
    try:
        model_data_preprocessing = data.copy(deep=True)
        model_data_preprocessing['sent_tokenized'] = model_data_preprocessing[
            'cleaned_text'].progress_apply(tokenize_sentence)

        model_data_preprocessing[
            'token_tag'] = model_data_preprocessing.progress_apply(
                lambda x: get_tags(sent=x['sent_tokenized'], rec_id=x['id'])
                if (isinstance(x['sent_tokenized'], list)) else np.nan, axis=1)

        model_data_preprocessing['vocab_sent_tokenized'], model_data_preprocessing[
            'sent_input_ids'], model_data_preprocessing[
                'vocab_token_tag'], model_data_preprocessing['token_tag_ids'] = zip(
                    *model_data_preprocessing.
                    progress_apply(lambda x: vocab_sent_tokenize_label(
                        sent_tokenized=x['sent_tokenized'], token_tag=x['token_tag'])
                                   if isinstance(x['token_tag'], list) else np.nan,
                                   axis=1))
        return model_data_preprocessing
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
def pad_data(input_ids, token_ids, max_token_length):
    try:
        attention_mask = []
        for input_ in tqdm(input_ids):
            attention_mask.append(torch.ones(len(input_[:max_token_length])))

        padded_attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask,
                                                                batch_first=True,
                                                                padding_value=0.0)

        padded_input_ids = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(input_[:max_token_length]) for input_ in input_ids],
            batch_first=True,
            padding_value=0.0)

        padded_tags = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(tag_[:max_token_length]) for tag_ in token_ids],
            batch_first=True,
            padding_value=0.0)
        return padded_input_ids, padded_attention_mask, padded_tags
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
def create_dataloader(token_ids, masks, tags, batch_size=16, val=False):
    try:
        # wrap tensors
        data = TensorDataset(token_ids, masks, tags)

        if val:
            # sampler for sampling the data during training
            sampler = SequentialSampler(data)
            
            # dataLoader for validation set
            dataloader = DataLoader(data,
                                    sampler=sampler,
                                    batch_size=batch_size)
        else:    
            # sampler for sampling the data during training
            sampler = RandomSampler(data)
            # dataLoader for train set
            dataloader = DataLoader(data,
                                    sampler=sampler,
                                    batch_size=batch_size)
        return dataloader
    except Exception as e:
        print(f"Error in line no: {sys.exc_info()[2].tb_lineno}")
        print(e)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('../input/longformer-files')

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
# function for evaluating the model
def do_evaluate(model, val_dataloader, loss_criteria):
    print("\nEvaluating...")
    
    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_logits = []

    # iterate over batches
    for step, batch in enumerate(val_dataloader):

        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            #             elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            logits = model(sent_id.to(device), mask.to(device))

            # compute the validation loss between actual and predicted values
            loss = loss_criteria(logits.logits.permute(0, 2, 1), labels)

            total_loss = total_loss + loss.item()

            logits = logits.logits.detach().cpu().numpy()

            total_logits.append(logits)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader)

    # reshape the predictions in form of (number of samples, no. of classes)
    total_logits = np.concatenate(total_logits, axis=0)

    return avg_loss, total_logits

In [ ]:
# Load fine-tuned model
model = AutoModelForTokenClassification.from_pretrained('../input/longformer-1000-seq-len-ner-model')
model = model.to(device)

In [ ]:
def get_prediction_from_logits(logits):
    try:
        tag_prob = nn.Softmax(dim=2)(logits)
        tag_prediction = torch.argmax(tag_prob, dim=2).detach().cpu().numpy()
        return tag_prediction
    except Exception as e:
        print(f"Error in line: {sys.exc_info()[2].tb_lineno}")
        print(e)
        
def classification_result(tag2idx, c_tag_id):
    try:
        prediction_result = []
        for sent_ in c_tag_id:
            prediction_result.append(
                list(map(lambda x: list(tag2idx.keys())[list(tag2idx.values()).index(x)], sent_))
            )
            
        tagged_entity = np.concatenate(prediction_result, axis=0)
        return tagged_entity
    except Exception as e:
        print(f"Error in line: {sys.exc_info()[2].tb_lineno}")
        print(e) 

# Predictions on Test data

In [ ]:
filepath = '../input/feedback-prize-2021/'
submission_data = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

In [ ]:
test_data_for_modelling = submission_data[['id']].drop_duplicates().reset_index(drop = True)
test_data_for_modelling['Text'] = np.NaN
for i in tqdm(range(0, test_data_for_modelling.shape[0])):
    record_id = test_data_for_modelling['id'].iloc[i]
    with open(filepath + 'test/' + record_id+'.txt','r') as f:
        data = f.read()
    test_data_for_modelling['Text'].iloc[i] = data
    
test_data_for_modelling['cleaned_text'] = test_data_for_modelling['Text'].apply(lambda x: re.sub('\n+',' ',x).strip().replace("\xa0"," "))

In [ ]:
print(test_data_for_modelling.shape)
test_data_for_modelling.head()

In [ ]:
# test_data_for_modelling = pd.concat([test_data_for_modelling]*2000, axis = 0).reset_index(drop = True)
# print(test_data_for_modelling.shape)
# test_data_for_modelling.head()

In [ ]:
processed_tagged_data = sent_tag_tokenization(data=test_data_for_modelling)

input_ids = processed_tagged_data['sent_input_ids'].tolist()
token_ids = processed_tagged_data['token_tag_ids'].tolist()

padded_input_ids, padded_attention_mask, padded_tags = pad_data(
    input_ids=input_ids, 
    token_ids=token_ids,
    max_token_length = 1000
)

batch_size = 50

test_dataloader = create_dataloader(
    token_ids=padded_input_ids, 
    masks=padded_attention_mask, 
    tags=padded_tags, 
    batch_size=batch_size, 
    val=True
)

pred_tags = []
for batch in tqdm(test_dataloader):
    # deactivate dropout layers
    model.eval()

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
        # model predictions
        logits = model(sent_id.to(device), mask.to(device))
        logits = get_prediction_from_logits(logits['logits'])
    pred_tags.append(logits)

In [ ]:
test_data_for_modelling['Predicted tags'] = 'to_be_filled_with_predicted_tags'
pred_tags = np.concatenate(pred_tags)
for i in range(0, len(test_data_for_modelling)):
    test_data_for_modelling.at[i, 'Predicted tags'] = np.array(pred_tags[i], dtype=object)
test_data_for_modelling['Predicted tags'] = test_data_for_modelling['Predicted tags'].apply(lambda x: [idx2tag[t] for t in x])

In [ ]:
print(test_data_for_modelling.shape)
test_data_for_modelling.head()

In [ ]:
# Get prediction string as per submission format
test_data_for_submission_final = pd.DataFrame(data = None, columns = ['id','class','predictionstring','token','discourse type count']) 
for i in tqdm(range(0, len(test_data_for_modelling))):
    test_data_for_submission = pd.DataFrame(data = None, columns = ['id','class','predictionstring','token']) 
    record_id = test_data_for_modelling['id'].iloc[i]
    pred_tags = test_data_for_modelling['Predicted tags'].iloc[i]
    record_text = test_data_for_modelling['cleaned_text'].iloc[i].split(' ')
    count = 0
    for j in range(0, len(pred_tags)):
        token = pred_tags[j]
        if j >= len(record_text):
            break
        if (token != 'O'):
            discourse_type = token.split('-')[1]
            test_data_for_submission.loc[count] = [record_id, discourse_type, count, token]
            count += 1
        else:
            discourse_type = np.NaN

    # Replace 2 adjacent B tags of same type together
    test_data_for_submission['actual token'] = test_data_for_submission['token']
    for j in range(1, len(test_data_for_submission)):
        if (test_data_for_submission['actual token'].iloc[j].split('-')[0] == 'B') & (test_data_for_submission['actual token'].iloc[j-1] == test_data_for_submission['actual token'].iloc[j]):
            test_data_for_submission['token'].iloc[j] = 'I-' + test_data_for_submission['token'].iloc[j].split('-')[1]
    
#     count = 0
#     test_data_for_submission['discourse type count'] = np.NaN
#     for k in range(0, len(test_data_for_submission)):
#         if test_data_for_submission['token'].iloc[k].split('-')[0] == 'B':
#             discourse_type = test_data_for_submission['token'].iloc[k].split('-')[1]
#             test_data_for_submission['discourse type count'].iloc[k] = count
#             for j in range(k+1, len(test_data_for_submission)):
#                 if (test_data_for_submission['token'].iloc[j].split('-')[0] == 'I') & (test_data_for_submission['token'].iloc[j].split('-')[1] == discourse_type):
#                     test_data_for_submission['discourse type count'].iloc[j] = count
#                 else:
#                     break
#             count = count + 1

    count = 0
    k = 0
    end_flag = False
    test_data_for_submission['discourse type count'] = np.NaN
    while (not (end_flag)):
        if test_data_for_submission['token'].iloc[k].split('-')[0] == 'B':
            discourse_type = test_data_for_submission['token'].iloc[k].split('-')[1]
            test_data_for_submission['discourse type count'].iloc[k] = count
            for j in range(k+1, len(test_data_for_submission)):
                if (test_data_for_submission['token'].iloc[j].split('-')[0] == 'I') & (test_data_for_submission['token'].iloc[j].split('-')[1] == discourse_type):
                    test_data_for_submission['discourse type count'].iloc[j] = count
                else:
                    k = j
                    break
            count = count + 1
        else:
            k = k + 1
        if (j== len(test_data_for_submission)-1) | (k== len(test_data_for_submission)-1):
            end_flag = True

    test_data_for_submission_final = pd.concat([test_data_for_submission_final, test_data_for_submission], axis = 0).reset_index(drop = True)
    
keys = test_data_for_submission_final['id'].unique().tolist()
values = [x for x in range(0, len(test_data_for_submission_final['id'].unique().tolist()))]
temp = dict(zip(keys, values))
test_data_for_submission_final['rec_num'] = test_data_for_submission_final['id'].map(temp) 

test_data_for_submission_final['predictionstring'] = test_data_for_submission_final['predictionstring'].astype(str)
test_data_for_submission_final = test_data_for_submission_final.groupby(['id','class','discourse type count','rec_num']).agg({'predictionstring': ' '.join}).reset_index().sort_values(by = ['rec_num','discourse type count'])[['id','class','predictionstring']]

In [ ]:
test_data_for_submission_final.head(10)

In [ ]:
test_data_for_submission_final.to_csv('submission.csv', index=False)